# Instalación de paquetes

In [39]:
!pip install selenium

In [40]:
!pip install selenium webdriver_manager

In [41]:
!apt-get update
!apt-get install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 8 no

# Librerías

In [42]:
import sys
import csv
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import urllib.parse
import json
import requests
from time import sleep
import pandas as pd

# Scrapping

## Funciones

In [45]:
def guardar_en_csv(csv_filename,json_arr):
  with open(csv_filename, 'w', newline='') as archivo_csv:
      # Define los nombres de las columnas (encabezados)
      nombres_columnas = json_arr[0].keys()

      # Crea un objeto escritor CSV
      escritor_csv = csv.DictWriter(archivo_csv, fieldnames=nombres_columnas)

      # Escribe los encabezados
      escritor_csv.writeheader()

      # Escribe los datos de cada fila
      for fila in json_arr:
          escritor_csv.writerow(fila)

  print(f"Los datos se han guardado en {csv_filename}")

In [46]:
def get_url_comment(user_id_div, referer):
  # UID_A2178A74F9FE3C13940A7A49162E85A7-SRC_919057329
  url_base = "https://www.tripadvisor.com.ar"
  split_user = user_id_div.split("_")
  source = split_user[2]
  user = split_user[1].split("-")[0]
  print("Usuario: ", user)
  print("Source: ", source)
  headers = {
      "authority": "www.tripadvisor.com.ar",
      "cache-control": "no-cache",
      "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
      "origin": "https://www.tripadvisor.com.ar",
      "pragma": "no-cache",
      "referer": referer,
      "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
      "x-requested-with": "XMLHttpRequest"
  }
  url = f"https://www.tripadvisor.com.ar/MemberOverlay?Mode=owa&uid={user}&c=&src={source}&fus=false&partner=false&LsoId=&metaReferer="
  data = {
    "haveJses": "earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Restaurant_Review,apg-Restaurant_Review-in,bootstrap,5xz,bt3,mqw,988,qlc,6nm,52f,xg7,yau,al4,gug,p5p,7th,nnp,ys6,eqo,9b9,ltl,xo1,4x6,dyf,qnd,fkj,jpp,qrk,q21,m49,u9j,gjt,0b0,90b,evv,gw8,2r4,mb2,frf,kq4,zq9,ess,lr6,904,b5n,eta,bxk,gbn,ltf,btb,nex,tm6,5gp,98i,8py,qjo,uvk,4eu,nnm,1ol,bvz,vjl,rdy,z2l,hxl,l9q,7u7,q13,uzm,qkj,gtt,65r,d8o,iog,eww,tp1,tbk,lch,xj2,3dy,vum,gyz,dsb,c3x,u5g,t9n,vbk,g05,vpo,s6q,ahl,fbu,3co,fgw,j8l,ule,mtb,32p,pro,q7w,45d,bqg,txq,fzz,bvv,4ax,3bt,w0d,w1v,qbn,ued,f7m,laj,7v9,0of,4q9,x9d,79a,jbz,ja5,d6v,ybx,tr3,sdp,skj,83p,973,6z5,t05,2mj,wr5,vop,o4y,teh,ifb,21h,l2u,8cr,5hc,ku7,ex1,pog,iyq,330,ag9,t5l,6g4,cny,8m6,80h,z6c,y52,k7d,665,yug,cyr,cje,wsk,dl4,76j,j56,1ua,j1i,uov,qx8,qhn,tf8,xnj,o3m,gbs,jnx,9yg,p4a,b1q,c5q,c5k,2l7,do5,kqf,s1y,0hk,5c8,con,7l5,flc,hwd,a6i,7gm,bp9,34b,fa3,w1y,r73,uc4,dlu,los,igy,jk7,1b5,uby,j8m,oz3,sw5,w5k,eoz,l75,zb2,nc3,0ad,y59,7ov,kkq,bke,g6k,b3v,sbz,00m,m5f,n6i,7g0,i9i,1ya,grn,siv,i3s,7nq,xm2,an5,rbx,p33,os3,yc7,cov,lt9,ar9,u3a,4il,i0z,my7,dvc,tj4,0sj,lyy,ntn,b1c,af7,43e,l47,4tv,24j,a2q,f2f,zuq,vfy,wrg,pg8,360,2w7,2mx,blj,d4c,aa8,7a7,a55,q6p,yw8,5zx,nm5,079,fug,u9y,mim,vws,lzo,l6h,8ec,pn8,gmr,mxu,2wl,qko,7s4,y0u,mat,xzn,d3i,55g,5h0,mc5,x0l,d5r,74j,7bj,s2g,mvp,wgu,nut,9l2,ec9,b49,vtj,o5y,3ai,p3y,bug,8fu,lji,35m,iun,w9f,ewp,q78,toc,vnf,vm2,rz5,9so,wzt,wiw,t5u,nnv,arw,wud,q1i,apa,1k8,r67,c38,yy6,l8y,rnf,11l,q32,o80,cgy,u6r,7dk,moe,wxj,9d7,ecg,zth,3r7,df0,508,yr8,n7m,kto,chl,bht,eun,b6u,9qb,us7,cmu,i2c,z2n,70j,55n,obr,ns8,v1g,2rn,yk7,p9u,joj,6m9,r6a,6ri,qk2,uw9,aj7,a85,9rb,tm4,qhc,050,2ws,irf,227,c44,he3,8ny,ams,4sf,kxi,ckg,mjd,25h,zto,56m,gtc,y5k,40q,w21,h9r,zsa,hu8,cjy,ymm,da2,77i,8sl,1u1,kgl,6zt,52b,ifp,all,nyl,7wb,06g,hdc,1f4,axz,2hd,eki,6q5,92b,ztz,w43,u47,n4z,weh,nnt,9w3,ahn,nds,amw,sa2,l56,jud,ibe,9d1,n48,9ir,kgv,2xm,vby,nj1,9m9,a3v,60m,7oy,2j5,g6h,jkv,ql7,96e,lya,q6i,fm6,us1,5xq,frb,57e,6mu,ifc,bhr,jgm,5wi,xv8,2t8,iwv,141,cby,dk1,0c1,z9k,hjf,wp1,rmu,2mm,ghg,q76,2jo,49c,acx,ljq,ay1,gud,x55,zj3,7ya,iwb,2he,iwu,dpz,96t,rfo,deg,asj,dpm,1gw,6qm,4wi,lsg,gx3,u59,94v,x4s,wxz,a79,cdp,tty,l71,m7w,cb7,qc8,943,ezm,9bt,o8g,fmn,bw5,2q3,3mq,mhk,8gg,2ve,qmi,b42,5i3,7j6,9en,nev,iw6,jz5,yji,yc8,vgf,r5t,4nz,5n3,633,epp,bck,lmi,wvn,tef,uea,q7j,gcj,zzf,rde,g5m,bqh,8gz,l82,6ac,r5w,crd,t9p,9l9,ctn,0xa,e1v,rod,r9z,btt,d0l,rko,ynq,s9v,inu,rbm,wt5,it9,k9d,qhd,k64,51q,hfp,3px,7yy,y2y,c24,bxt,i4i,otn,lm2,kww,jo0,k3t,wzx,89z,2z6,ptd,gmg,k5n,4ec,3w1,j9p,t73,tsg,mhd,8vo,3ky,ura,0l1,ptg,itm,ure,adx,nj5,4s1,4f7,vmd,ilf,hmc,0tr,sqi,nv3,ko0,nv2,68f,1vx,7qv,v5c,kpl,shk,38p,bdd,uu0,5z3,xba,sfn,j0c,5nt,ibn,7sf,lw6,sev,zuz,tl3,fbj,7f4,azy,arz,4z4,yld,6u6,d8i,e2v,d0n,pl5,u67,2wh,ue4,nls,93t,wrd,1bl,bzs,gnd,b47,c2b,69z,ek9,2ko,dsz,8cx,paf,47l,ogf,v6m,kgn,j8u,bov,lq6,6hf,n1m,c96,bj0,z2i,468,5hn,2qd,ego,w1k,fy4,z72,a7a,0cw,dvd,cbd,kqy,yer,kdt,a34,75v,q18,556,5az,it6,ue3,008,0v7,ofj,cvh,w2p,gu0,masthead_search_late_load,p13n_masthead_search__deferred__lateHandlers",
    "Action": "install"
  }
  timeout = (10, 20)
  max_retries = 3  # Número máximo de intentos
  retry_count = 0
  while retry_count < max_retries:
    try:
      response = requests.post(url, headers=headers, data=json.dumps(data), timeout=timeout)
      break
    except requests.exceptions.Timeout:
      print("La solicitud ha superado el tiempo límite.")
    except requests.exceptions.RequestException as e:
      print(f"Ocurrió un error al realizar la solicitud: {e}")
    retry_count += 1
    print(f"Reintentando ({retry_count}/{max_retries})...")
  if retry_count == max_retries:
    print("Se ha alcanzado el número máximo de intentos.")
    return None
  if response.status_code == 200:
    html = response.text
    soup_user = BeautifulSoup(html, "html.parser")
    div_profile = soup_user.find("div",class_="memberOverlayRedesign g10n").find("a")['href']
    url_profile = url_base + div_profile
    #print(url_profile)
    return url_profile

    #print(soup.prettify())
  else:
    print("La solicitud no fue exitosa. Código de respuesta:", response.status_code)
    print(response.text)
    return None

## Configuración

In [47]:
# default tripadvisor website of restaurant
url = "https://www.tripadvisor.com.ar/Restaurants-g312741-Buenos_Aires_Capital_Federal_District.html"
url = "https://www.tripadvisor.com.ar/Restaurants-g312741-c8-Buenos_Aires_Capital_Federal_District.html"
url = "https://www.tripadvisor.com.ar/Restaurants-g312740-zfg9900-Capital_Federal_District.html"
#url = "https://www.tripadvisor.com.ar/RestaurantSearch-g312741-c10642-Buenos_Aires_Capital_Federal_District.html"
# if you pass the inputs in the command line
if (len(sys.argv) == 4):
    path_to_file = sys.argv[1]
    num_page = int(sys.argv[2])
    url = sys.argv[3]

In [48]:
# Import the webdriver for chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

service = ChromeService(executable_path='./chromedriver.exe')
print(service)
driver=webdriver.Chrome(options=chrome_options)
driver.get(url)

In [49]:
soup = BeautifulSoup(driver.page_source)
print(soup)

<html lang="es-AR"><head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><meta content="es" http-equiv="content-language"/><link as="font" crossorigin="anonymous" href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans-VF.woff2?v1.002" rel="preload" type="font/woff2"/><link href="https://static.tacdn.com/favicon.ico?v2" id="favicon" rel="icon" type="image/x-icon"/><link color="#000000" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" rel="mask-icon" sizes="any"/><meta content="#34e0a1" name="theme-color"/><meta content="telephone=no" name="format-detection"/><meta content="TripAdvisor" property="al:ios:app_name"/><meta content="284876795" property="al:ios:app_store_id"/><meta content="284876795" name="twitter:app:id:ipad" property="twitter:app:id:ipad"/><meta content="284876795" name="twitter:app:id:iphone" property="twitter:app:id:iphone"/><meta content="tripadvisor://www.tripadvisor.com.ar/Restaurants-g312740-zfg9900-Capita

In [ ]:
num_restaurant = int(soup.find("div",class_="LftMO").find('span', class_='b').text)
print(num_restaurant)

572


In [50]:
urls = []
url = "https://www.tripadvisor.com.ar/Restaurants-g312741-Buenos_Aires_Capital_Federal_District.html"
url = "https://www.tripadvisor.com.ar/RestaurantSearch-g312741-c10642-Buenos_Aires_Capital_Federal_District.html"
url = "https://www.tripadvisor.com.ar/Restaurants-g312741-c8-Buenos_Aires_Capital_Federal_District.html"
url = "https://www.tripadvisor.com.ar/Restaurants-g312740-zfg9900-Capital_Federal_District.html"
for num in range(0, num_restaurant + 1, 30):
  if num != 0:
    start_pos = url.find("g312740-")+len("g312740-")
    end_pos = url.find("-zfg9900")
    url = url[0:start_pos]+'oa'+str(num)+url[end_pos:]
  urls.append(url)

In [51]:
len(urls)

20

In [52]:
urls

['https://www.tripadvisor.com.ar/Restaurants-g312740-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa30-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa60-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa90-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa120-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa150-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa180-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa210-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa240-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa270-zfg9900-Capital_Federal_District.html',
 'https://www.tripa

In [ ]:
restaurants_=[]

In [ ]:
for url in urls:
  driver.get(url)
  print(f"URL BÚSQUEDA: {url}")
  soup = BeautifulSoup(driver.page_source)
  restaurants = soup.find_all("div",class_="VDEXx u Ff K")
  for restaurant in restaurants:
    #Obtener la URL de cada Restaurant para ir accediendo al sitio
    url_restaurant = "https://www.tripadvisor.com.ar"+restaurant.find("a")['href']
    start_pos = url_restaurant.find("g312741-d")+len("g312741-d")
    end_pos = url_restaurant.find("-Reviews-")
    restaurant_id = url_restaurant[start_pos:end_pos]
    driver.get(url_restaurant)
    soup_restaurant = BeautifulSoup(driver.page_source)
    text = soup_restaurant.find_all('script')
    for t in text:
      if "window.__WEB_CONTEXT__" in t.text:
        result=t.text.replace("window.__WEB_CONTEXT__=","").replace("{pageManifest:","")
        end_pos=result.find("};")
        json_=json.loads(result[0:end_pos])
        index=f"/data/1.0/location/{restaurant_id}"
        try:
          data=json_["redux"]["api"]["responses"][index]["data"]
        except:
          continue
        try:
          email=data["email"]
        except:
          email=''
        try:
          phone=data["phone"]
        except:
          phone=''
        try:
          imagen=data["photo"]["images"]["original"]["url"]
        except:
          imagen=''
        try:
          website=data['website']
        except:
          website=''
        try:
          latitude=data["latitude"]
          longitude=data["longitude"]
        except:
          latitude=''
          longitude=''
        try:
          name=data["name"]
        except:
          continue
        try:
          timezone=data["timezone"]
        except:
          timezone=''
        try:
          location=data["location_string"]
        except:
          location=''
        try:
          raw_ranking=data["raw_ranking"]
        except:
          raw_ranking=''
        try:
          ranking_position=data["ranking_position"]
        except:
          ranking_position=''
        try:
          rating=data["rating"]
        except:
          rating=''
        try:
          price_level=data["price_level"]
        except:
          price_level=''
        try:
          address=data["address"]
        except:
          address=''
        try:
          postalcode=data["address_obj"]["postalcode"]
        except:
          postalcode=''
        try:
          location_id=data["location_id"]
        except:
          location_id=''
        try:
          num_reviews=data["num_reviews"]
        except:
          num_reviews=''
        try:
          display_hours=data["display_hours"]
        except:
          display_hours=''
        index=f"/data/1.0/restaurant/{restaurant_id}/overview"
        try:
          data=json_["redux"]["api"]["responses"][index]["data"]
        except:
          continue
        #print(data)
        try:
          ranking=data["rating"]["primaryRanking"]["rank"]
        except:
          ranking=''

    all_types = soup_restaurant.find_all("a",class_="dlMOJ")
    restaurant_type = []
    for one_type in all_types:
      text = one_type.text
      restaurant_type.append(text)

    titles = soup_restaurant.find_all("div",class_="tbUiL b")
    values = soup_restaurant.find_all("div",class_="SrqKb")
    special_diets=''
    pricing=''
    for i in range(len(titles)):
      if titles[i].text.upper() == "DIETAS ESPECIALES":
        special_diets = values[i].text
      if titles[i].text.upper() == "RANGO DE PRECIOS":
        pricing = values[i].text

    califications_ = {}
    califications = soup_restaurant.find_all("div",class_="DzMcu")
    for c in califications:
      calification = c.find("span",class_="BPsyj").text
      value = int(c.find("span",class_="ui_bubble_rating")['class'][1].split("_")[1])/10
      califications_[calification] = value

    traveler_rating_ = {}
    all_traveler_rating = soup_restaurant.find_all("div",class_="ui_checkbox item")
    for tr in all_traveler_rating:
      traveler_rating = tr.find("label",class_="row_label label no_display")
      number = tr.find("span",class_="row_num")
      if traveler_rating is not None:
        traveler_rating_[traveler_rating.text] = number.text

    restaurants_.append({
      "id": location_id,
      "name": name,
      "num_reviews": num_reviews,
      "latitude": latitude,
      "longitude": longitude,
      "timezone": timezone,
      "location": location,
      "raw_ranking": raw_ranking,
      "ranking_position": ranking_position,
      "ranking_caba": ranking,
      "rating": rating,
      "price": pricing,
      "price_level": price_level,
      "phone": phone,
      "website": website,
      "email": email,
      "address": address,
      "postalcode": postalcode,
      "image_url": imagen,
      "type": restaurant_type,
      "url": url_restaurant,
      "special_diets": special_diets,
      "califications": califications_,
      "traveler_rating": traveler_rating_,
      "display_hours": display_hours
    })

    #ranking=data["ranking_position"]
    print(f"URL RESTAURANT {ranking}: {url_restaurant}")
    #break
  #break


URL BÚSQUEDA: https://www.tripadvisor.com.ar/Restaurants-g312740-zfg9900-Capital_Federal_District.html
URL RESTAURANT 1: https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d10048394-Reviews-Vive_Cafe-Buenos_Aires_Capital_Federal_District.html
URL RESTAURANT 2: https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d2424768-Reviews-Full_City_Coffee_House-Buenos_Aires_Capital_Federal_District.html
URL RESTAURANT 8: https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d1902671-Reviews-Havanna-Buenos_Aires_Capital_Federal_District.html
URL RESTAURANT 4: https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d3676783-Reviews-El_Gato_Negro-Buenos_Aires_Capital_Federal_District.html
URL RESTAURANT 5: https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d6563785-Reviews-Petit_Colon-Buenos_Aires_Capital_Federal_District.html
URL RESTAURANT 3: https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d3708642-Reviews-Cocu_Boulangerie-Buenos_Aires_Capital_Federal_District.html
UR

In [ ]:
restaurants_

[{'id': '10048394',
  'name': 'Vive Cafe',
  'num_reviews': '357',
  'latitude': '-34.58184',
  'longitude': '-58.436085',
  'timezone': 'America/Argentina/Buenos_Aires',
  'location': 'Buenos Aires, Distrito Capital Federal',
  'raw_ranking': '4.716580390930176',
  'ranking_position': '21',
  'ranking_caba': 1,
  'rating': '5.0',
  'price': 'US$\xa02 - US$\xa09',
  'price_level': '$',
  'phone': '+54 11 4774-5461',
  'website': 'http://instagram.com/vivecafe?igshid=NzZlODBkYWE4Ng==',
  'email': 'vivecafe@outlook.com',
  'address': 'Costa Rica 5722, Buenos Aires 1414 Argentina',
  'postalcode': '1414',
  'image_url': 'https://media-cdn.tripadvisor.com/media/photo-o/19/2e/37/01/photo2jpg.jpg',
  'type': ['$', 'Café', 'Colombiana', 'Apto para vegetarianos'],
  'url': 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d10048394-Reviews-Vive_Cafe-Buenos_Aires_Capital_Federal_District.html',
  'special_diets': 'Apto para vegetarianos, Opciones veganas, Opciones sin gluten',
  'califi

In [ ]:
restaurants

[<div class="VDEXx u Ff K"><div class="biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"><a class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS" href="/Restaurant_Review-g8838820-d26440114-Reviews-Cazara_Arte_Cafe-Banfield_Lomas_de_Zamora_Province_of_Buenos_Aires_Central_Arge.html" target="_blank">571<!-- -->. <!-- -->Cazara  Arte &amp; Café </a></div></div>,
 <div class="VDEXx u Ff K"><div class="biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"><a class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS" href="/Restaurant_Review-g8078335-d14081831-Reviews-Buena_Vista-Muniz_Province_of_Buenos_Aires_Central_Argentina.html" target="_blank">572<!-- -->. <!-- -->Buena Vista</a></div></div>]

In [ ]:
restaurants_

[{'id': '10048394',
  'name': 'Vive Cafe',
  'num_reviews': '352',
  'latitude': '-34.58184',
  'longitude': '-58.436085',
  'timezone': 'America/Argentina/Buenos_Aires',
  'location': 'Buenos Aires, Distrito Capital Federal',
  'raw_ranking': '4.7084550857543945',
  'ranking_position': '21',
  'ranking_caba': 1,
  'rating': '5.0',
  'price': 'US$\xa00 - US$\xa02',
  'price_level': '$',
  'phone': '+54 11 4774-5461',
  'website': 'http://www.facebook.com/ViveCafeTienda',
  'email': 'vivecafe@outlook.com',
  'address': 'Costa Rica 5722, Buenos Aires 1414 Argentina',
  'postalcode': '1414',
  'image_url': 'https://media-cdn.tripadvisor.com/media/photo-o/19/2e/37/01/photo2jpg.jpg',
  'type': ['$', 'Café', 'Colombiana', 'Apto para vegetarianos'],
  'url': 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d10048394-Reviews-Vive_Cafe-Buenos_Aires_Capital_Federal_District.html',
  'special_diets': 'Apto para vegetarianos, Opciones veganas, Opciones sin gluten',
  'califications': {'Co

In [ ]:
len(restaurants_)

0

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
guardar_en_csv('/content/drive/MyDrive/MaestriaDataMining/SistemaDeRecomendación/Scraping/restaurants_cafe.csv',restaurants_)

Los datos se han guardado en /content/drive/MyDrive/MaestriaDataMining/SistemaDeRecomendación/Scraping/restaurants_cafe.csv


In [54]:
interacciones =[]
usuarios =[]

In [55]:
interacciones

[]

In [56]:
csv_interacciones='/content/drive/MyDrive/MaestriaDataMining/SistemaDeRecomendación/Scraping/interacciones_cafe.csv'
csv_usuarios='/content/drive/MyDrive/MaestriaDataMining/SistemaDeRecomendación/Scraping/usuarios_cafe.csv'
csv_urls='/content/drive/MyDrive/MaestriaDataMining/SistemaDeRecomendación/Scraping/url_cafe.csv'

In [57]:
fieldnames_interacciones = ["restaurant_id","user_id","rating","rating_date","title","opinion"]
fieldnames_usuarios = ["user_id","nick","user","locate","num_opinions","total_points","helpful_counts","critical_level"]
fieldnames_urls = ["id","url"]

In [ ]:
with open(csv_interacciones, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames_interacciones)
    writer.writeheader()

with open(csv_usuarios, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames_usuarios)
    writer.writeheader()

with open(csv_urls, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames_urls)
    writer.writeheader()

In [58]:
df_restaurant_recorridos = pd.read_csv("/content/drive/MyDrive/MaestriaDataMining/SistemaDeRecomendación/Scraping/url_cafe.csv")
df_restaurant_recorridos = df_restaurant_recorridos.iloc[:-1]
restaurant_recorridos = df_restaurant_recorridos['url'].tolist()
restaurant_recorridos

['https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d10048394-Reviews-Vive_Cafe-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d2424768-Reviews-Full_City_Coffee_House-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d1902671-Reviews-Havanna-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d1902671-Reviews-Havanna-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d3676783-Reviews-El_Gato_Negro-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d6563785-Reviews-Petit_Colon-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d3708642-Reviews-Cocu_Boulangerie-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d

In [59]:

restaurant_recorridos

['https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d10048394-Reviews-Vive_Cafe-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d2424768-Reviews-Full_City_Coffee_House-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d1902671-Reviews-Havanna-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d1902671-Reviews-Havanna-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d3676783-Reviews-El_Gato_Negro-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d6563785-Reviews-Petit_Colon-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d3708642-Reviews-Cocu_Boulangerie-Buenos_Aires_Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d

In [60]:
for url in urls:
  driver.get(url)
  print(f"URL BÚSQUEDA: {url}")
  soup = BeautifulSoup(driver.page_source)
  restaurants = soup.find_all("div",class_="VDEXx u Ff K")
  for restaurant in restaurants:
    #Obtener la URL de cada Restaurant para ir accediendo al sitio
    url_restaurant = "https://www.tripadvisor.com.ar"+restaurant.find("a")['href']
    print(f"URL RESTAURANT: {url_restaurant}")
    if url_restaurant in restaurant_recorridos:
      print(f"Ya se tienen los datos de {url_restaurant}")
      continue

    start_pos = url_restaurant.find("g312741-d")+len("g312741-d")
    end_pos = url_restaurant.find("-Reviews-")
    restaurant_id = url_restaurant[start_pos:end_pos]
    driver.get(url_restaurant)
    soup_restaurant = BeautifulSoup(driver.page_source)
    url_json = {
      "id": restaurant_id,
      "url": url_restaurant
    }
    with open(csv_urls, 'a', newline='', encoding='utf-8') as csvfile_urls:
        writer_url = csv.DictWriter(csvfile_urls, fieldnames=fieldnames_urls)
        writer_url.writerow(url_json)
    try:
      num_reviews = int(soup_restaurant.find("label",{"for": "filters_detail_language_filterLang_es"}).find("span",class_="count").text.replace('(','').replace(')','').replace('.',''))
    except Exception as e:
      num_reviews = 0
    print(f"NUMBER OF REVIEWS: {num_reviews}")
    if num_reviews > 2000:
      num_reviews = 2000
    url_reviews=url_restaurant
    #break
    for num in range(0, num_reviews + 1, 10):
      if num != 0:
        start_pos = url_restaurant.find("-Reviews-")+len("-Reviews-")
        url_reviews = url_restaurant[0:start_pos]+'or'+str(num)+'-'+url_restaurant[start_pos:]
      print(f"URL REVIEWS: {url_reviews}")
      driver.get(url_reviews)
      soup_reviews = BeautifulSoup(driver.page_source)
      reviews = soup_reviews.find_all("div",class_="rev_wrap ui_columns is-multiline")
      for review in reviews:
        time.sleep(2)
        user = review.find("div",class_='info_text pointer_cursor')

        if user is not None:
          user_div = review.find("div",class_='memberOverlayLink clickable')
          user_id_div = user_div.get('id')
          url_profile = get_url_comment(user_id_div, driver.current_url)
          if url_profile is None:
            continue
          elif 'MemberProfile-a_uid' in url_profile:
            continue
          user_id = user_id_div.split('-')[0].split('_')[1]
          print(f"URL PROFILE: {url_profile}")
          driver.get(url_profile)
          soup_profile = BeautifulSoup(driver.page_source)
          try:
            user_name = soup_profile.find("span",class_="OUDwj b brsfY").text
          except:
            user_name = url_profile.split('/')[4]
          try:
            nick_name = soup_profile.find("span",class_="Dsdjn _R").text
          except:
            nick_name = url_profile.split('/')[4]
          try:
            user_locate = soup_profile.find("span",class_="PacFI _R S4 H3 LXUOn default").text
          except:
            user_locate = ''
          try:
            num_opinions = int(review.find("span",class_='badgeText').text.split(' ')[0])
          except:
            num_opinions = 1
          user = user.text
          user_rating = int(review.find("span",class_='ui_bubble_rating')['class'][1].split("_")[1])/10
          rating_date = review.find("span",class_='ratingDate')['title']
          title = review.find("span",class_='noQuotes').text
          opinion = review.find("p",class_='partial_entry').text

          interaccion_json = {
              "restaurant_id": restaurant_id,
              "user_id": user_id,
              "rating": user_rating,
              "rating_date": rating_date,
              "title": title,
              "opinion": opinion
          }

          interacciones.append(interaccion_json)
          intentos = 0
          write = False
          while intentos < 3 and write == False:
            try:
              with open(csv_interacciones, 'a', newline='', encoding='utf-8') as csvfile_interacciones:
                writer_interacciones = csv.DictWriter(csvfile_interacciones, fieldnames=fieldnames_interacciones)
                writer_interacciones.writerow(interaccion_json)
                write = True
            except Exception as e:
              intentos = intentos + 1
          if not write:
            print(f"Error al escribir la interacción {restaurant_id} y {user_id}")

          url_comunidadtrip = f'https://www.tripadvisor.com.ar/members-badgecollection/{nick_name}'
          print(f'URL COMUNIDAD TRIP: {url_comunidadtrip}')
          driver.get(url_comunidadtrip)
          soup_comunidadtrip = BeautifulSoup(driver.page_source)
          try:
            total_points = int(soup_comunidadtrip.find("div",class_="modules-membercenter-total-points").find("div",class_="points").text.strip())
          except:
            total_points = 0
          try:
            helpful_counts = int(soup_comunidadtrip.find("div",class_="modules-membercenter-helpful-counts").find("div",class_="points").text.strip())
          except:
            helpful_counts = 0
          try:
            critical_level = int(soup_comunidadtrip.find("div",class_="level tripcollectiveinfo").find("span").text)
          except:
            critical_level = 0

          usuario_json = {
              "user_id": user_id,
              "nick": nick_name,
              "user": user_name,
              "locate": user_locate,
              "num_opinions": num_opinions,
              "total_points": total_points,
              "helpful_counts": helpful_counts,
              "critical_level": critical_level
          }
          usuarios.append(usuario_json)
          intentos = 0
          write = False
          while intentos < 3 and write == False:
            try:
              with open(csv_usuarios, 'a', newline='', encoding='utf-8') as csvfile_usuarios:
                writer_usuarios = csv.DictWriter(csvfile_usuarios, fieldnames=fieldnames_usuarios)
                writer_usuarios.writerow(usuario_json)
                write = True
            except Exception as e:
              intentos = intentos + 1
          if not write:
            print(f"Error al escribir el usuario {user_id}")

    time.sleep(1)


Se truncaron las últimas líneas 5000 del resultado de transmisión.
Usuario:  9ACD7FD2896080880E13A0D92C5446B2
Source:  744658646
URL PROFILE: https://www.tripadvisor.com.ar/Profile/florenciadm
URL COMUNIDAD TRIP: https://www.tripadvisor.com.ar/members-badgecollection/florenciadm
Usuario:  5E0AF4E055C1B1FC2A56570B49E86FF6
Source:  709988705
URL PROFILE: https://www.tripadvisor.com.ar/Profile/Cecieraso
URL COMUNIDAD TRIP: https://www.tripadvisor.com.ar/members-badgecollection/Cecieraso
Usuario:  ABAE462A1BB0F1B63A4BCCA30A6EB551
Source:  704410431
URL PROFILE: https://www.tripadvisor.com.ar/Profile/llovera11
URL COMUNIDAD TRIP: https://www.tripadvisor.com.ar/members-badgecollection/llovera11
Usuario:  60008ADFC778C764FD834EE4CBE29CE5
Source:  673535496
URL PROFILE: https://www.tripadvisor.com.ar/Profile/beleishul
URL COMUNIDAD TRIP: https://www.tripadvisor.com.ar/members-badgecollection/beleishul
Usuario:  EE2379BC1E7020800F1CE01D0D22308D
Source:  673485422
URL PROFILE: https://www.tripad

In [61]:
urls

['https://www.tripadvisor.com.ar/Restaurants-g312740-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa30-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa60-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa90-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa120-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa150-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa180-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa210-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa240-zfg9900-Capital_Federal_District.html',
 'https://www.tripadvisor.com.ar/Restaurants-g312740-oa270-zfg9900-Capital_Federal_District.html',
 'https://www.tripa

In [ ]:
for url in urls:
  driver.get(url)
  print(f"URL BÚSQUEDA: {url}")
  soup = BeautifulSoup(driver.page_source)
  restaurants = soup.find_all("div",class_="VDEXx u Ff K")
  for restaurant in restaurants:
    #Obtener la URL de cada Restaurant para ir accediendo al sitio
    url_restaurant = "https://www.tripadvisor.com.ar"+restaurant.find("a")['href']
    print(f"URL RESTAURANT: {url_restaurant}")
    start_pos = url_restaurant.find("g312741-d")+len("g312741-d")
    end_pos = url_restaurant.find("-Reviews-")
    restaurant_id = url_restaurant[start_pos:end_pos]
    driver.get(url_restaurant)
    soup_restaurant = BeautifulSoup(driver.page_source)

    num_reviews = int(soup_restaurant.find("label",{"for": "filters_detail_language_filterLang_es"}).find("span",class_="count").text.replace('(','').replace(')','').replace('.',''))
    print(f"NUMBER OF REVIEWS: {num_reviews}")
    url_reviews=url_restaurant
    #break
    for num in range(0, num_reviews + 1, 10):
      if num != 0:
        start_pos = url_restaurant.find("-Reviews-")+len("-Reviews-")
        url_reviews = url_restaurant[0:start_pos]+'or'+str(num)+'-'+url_restaurant[start_pos:]
      print(f"URL REVIEWS: {url_reviews}")
      driver.get(url_reviews)
      soup_reviews = BeautifulSoup(driver.page_source)
      reviews = soup_reviews.find_all("div",class_="rev_wrap ui_columns is-multiline")
      for review in reviews:
        time.sleep(2)
        user = review.find("div",class_='info_text pointer_cursor')

        if user is not None:
          user_div = review.find("div",class_='memberOverlayLink clickable')
          user_id_div = user_div.get('id')
          url_profile = get_url_comment(user_id_div, driver.current_url)
          if url_profile is None:
            continue
          elif 'MemberProfile-a_uid' in url_profile:
            continue
          user_id = user_id_div.split('-')[0].split('_')[1]
          print(f"URL PROFILE: {url_profile}")
          driver.get(url_profile)
          soup_profile = BeautifulSoup(driver.page_source)
          try:
            user_name = soup_profile.find("span",class_="OUDwj b brsfY").text
          except:
            user_name = url_profile.split('/')[4]
          try:
            nick_name = soup_profile.find("span",class_="Dsdjn _R").text
          except:
            nick_name = url_profile.split('/')[4]
          try:
            user_locate = soup_profile.find("span",class_="PacFI _R S4 H3 LXUOn default").text
          except:
            user_locate = ''
          try:
            num_opinions = int(review.find("span",class_='badgeText').text.split(' ')[0])
          except:
            num_opinions = 1
          user = user.text
          user_rating = int(review.find("span",class_='ui_bubble_rating')['class'][1].split("_")[1])/10
          rating_date = review.find("span",class_='ratingDate')['title']
          title = review.find("span",class_='noQuotes').text
          opinion = review.find("p",class_='partial_entry').text

          interacciones.append({
              "restaurant_id": restaurant_id,
              "user_id": user_id,
              "rating": user_rating,
              "rating_date": rating_date,
              "title": title,
              "opinion": opinion
          })

          url_comunidadtrip = f'https://www.tripadvisor.com.ar/members-badgecollection/{nick_name}'
          print(f'URL COMUNIDAD TRIP: {url_comunidadtrip}')
          driver.get(url_comunidadtrip)
          soup_comunidadtrip = BeautifulSoup(driver.page_source)
          try:
            total_points = int(soup_comunidadtrip.find("div",class_="modules-membercenter-total-points").find("div",class_="points").text.strip())
          except:
            total_points = 0
          try:
            helpful_counts = int(soup_comunidadtrip.find("div",class_="modules-membercenter-helpful-counts").find("div",class_="points").text.strip())
          except:
            helpful_counts = 0
          try:
            critical_level = int(soup_comunidadtrip.find("div",class_="level tripcollectiveinfo").find("span").text)
          except:
            critical_level = 0

          usuarios.append({
              "user_id": user_id,
              "nick": nick_name,
              "user": user_name,
              "locate": user_locate,
              "num_opinions": num_opinions,
              "total_points": total_points,
              "helpful_counts": helpful_counts,
              "critical_level": critical_level
          })

    time.sleep(1)


Se truncaron las últimas líneas 5000 del resultado de transmisión.
Source:  475414116
URL PROFILE: https://www.tripadvisor.com.ar/Profile/Aleamunina
URL COMUNIDAD TRIP: https://www.tripadvisor.com.ar/members-badgecollection/Aleamunina
Usuario:  116B390AE7C5D7698E5EBBE53F799484
Source:  475274549
URL PROFILE: https://www.tripadvisor.com.ar/Profile/765magdas
URL COMUNIDAD TRIP: https://www.tripadvisor.com.ar/members-badgecollection/765magdas
URL REVIEWS: https://www.tripadvisor.com.ar/Restaurant_Review-g312741-d806778-Reviews-or1990-Cafe_Tortoni-Buenos_Aires_Capital_Federal_District.html
Usuario:  B907D81EEFF2541A429206C4E058A202
Source:  475134418
URL PROFILE: https://www.tripadvisor.com.ar/Profile/dahiannam2017
URL COMUNIDAD TRIP: https://www.tripadvisor.com.ar/members-badgecollection/dahiannam2017
Usuario:  2AB28CB904523FAC028F0D5240249D7D
Source:  475109162
URL PROFILE: https://www.tripadvisor.com.ar/Profile/sof_ap299
URL COMUNIDAD TRIP: https://www.tripadvisor.com.ar/members-badgeco

In [ ]:
len(interacciones)

0

In [ ]:
interacciones

In [ ]:
len(usuarios)

0

In [ ]:
usuarios

In [ ]:
guardar_en_csv('usuarios.csv',usuarios)
guardar_en_csv('interacciones.csv',interacciones)